# Lexicon Generation

The objective is to identify relevant words for finacial markets. 
To achieve this, 



- Objectif : Identifier les mots qui ont un impact réel sur le marché financier.
- Procédure : 
    - Prendre les articles de presse sur une fenêtre de 4 semaines précédant le jour $d$.
    - Calculer la corrélation de chaque mot avec la variation du prix de l'indice ($\Delta$) le jour suivant sa publication.
    - Filtrage des mots trop fréquents (>90% des documents) ou trop rares (<10 documents).
    - Sélection des mots situés dans les percentiles extrêmes (les plus positifs et les plus négatifs) pour former le lexique "conscient du temps".

In [19]:
import pandas as pd
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import re
from datetime import datetime, timedelta
from gensim.parsing import preprocessing as pproc

In [ ]:


# --- 1. NETTOYAGE GLOBAL (VITESSE OPTIMISÉE) ---
def clean_text_authors(text):
    if not isinstance(text, str) or len(text) < 20:
        return ""
    text = text.lower()
    # Retrait abréviations
    text = re.sub(r'(?:[a-z]\.)+', lambda m: m.group().replace('.', ''), text)
    # Gensim strip punctuation & stopwords
    text = pproc.strip_punctuation(text)
    text = pproc.remove_stopwords(text)
    # Filtre alphabétique strict (retrait du bruit numérique '05pm', etc.)
    tokens = [w for w in text.split() if w.isalpha() and len(w) > 2]
    # Stemming
    return " ".join([pproc.stem_text(w) for w in tokens])

# --- 2. FONCTION DE RECHERCHE DU DELTA (WEEK-ENDS) ---
def get_next_delta(pub_date, market_dict, max_days=4):
    for i in range(1, max_days + 1):
        next_d = pub_date + timedelta(days=i)
        if next_d in market_dict:
            return market_dict[next_d]
    return None

# --- 3. GÉNÉRATION DU LEXIQUE POUR TOUTE LA PÉRIODE ---
def generate_full_period_lexicons(news_path, price_path):
    # A. Chargement des News
    print("1. Chargement et nettoyage des news (patience...)")
    df_news = pd.read_csv(news_path).drop_duplicates(subset=['headline'])
    df_news['date_dt'] = pd.to_datetime(df_news['date_gdelt'].astype(str).str[:8], format='%Y%m%d').dt.date
    # On nettoie tout une seule fois ici
    tqdm.pandas()
    df_news['clean_text'] = (df_news['headline'].fillna('') + " " + df_news['body'].fillna('')).progress_apply(clean_text_authors)

    # B. Chargement des Prix
    print("2. Chargement des prix et calcul des Delta...")
    df_price = pd.read_csv(price_path, skiprows=3, names=['Date', 'Close', 'High', 'Low', 'Open', 'Volume'])
    df_price['Date'] = pd.to_datetime(df_price['Date']).dt.date
    df_price = df_price.dropna(subset=['Close']).sort_values('Date')
    df_price['Delta'] = df_price['Close'].pct_change()
    market_dict = df_price.set_index('Date')['Delta'].to_dict()

    # C. Définition de la période (Départ le 1er Février 2019 pour avoir 28j d'historique)
    start_date = datetime(2019, 2, 1).date()
    end_date = df_price['Date'].max()
    
    all_daily_lexicons = [] # On va stocker ça en liste pour faire un CSV à la fin

    print(f"3. Calcul des lexiques quotidiens de {start_date} à {end_date}...")
    pbar = tqdm(total=(end_date - start_date).days + 1)
    
    current_date = start_date
    while current_date <= end_date:
        # Fenêtre glissante [D-28, D-1]
        window_news = df_news[(df_news['date_dt'] >= (current_date - timedelta(days=28))) & 
                              (df_news['date_dt'] < current_date)].copy()
        
        if len(window_news) >= 10:
            # Attribution du delta impact d+1
            window_news['y'] = window_news['date_dt'].apply(lambda d: get_next_delta(d, market_dict))
            window_news = window_news.dropna(subset=['y'])
            
            if not window_news.empty:
                # TF-IDF Binaire
                vectorizer = TfidfVectorizer(max_df=0.9, min_df=5, binary=True, use_idf=False, norm=None)
                try:
                    X = vectorizer.fit_transform(window_news['clean_text'])
                    words = vectorizer.get_feature_names_out()
                    
                    # Multiplication par Delta
                    deltas = window_news['y'].values
                    for i in range(X.shape[0]):
                        X.data[X.indptr[i]:X.indptr[i+1]] *= deltas[i]
                    
                    # Score f(j)
                    counts = np.array(X.getnnz(axis=0))
                    counts[counts == 0] = 1
                    scores = np.asarray(X.sum(axis=0)).flatten() / counts
                    
                    # Screening Percentiles 80/20
                    p80, p20 = np.percentile(scores, 80), np.percentile(scores, 20)
                    
                    pos_words = [words[i] for i in range(len(words)) if scores[i] >= p80 and scores[i] > 0]
                    neg_words = [words[i] for i in range(len(words)) if scores[i] <= p20 and scores[i] < 0]
                    
                    # Sauvegarde pour ce jour
                    all_daily_lexicons.append({
                        'date': current_date,
                        'positive_words': ",".join(pos_words),
                        'negative_words': ",".join(neg_words)
                    })
                except: pass
        
        current_date += timedelta(days=1)
        pbar.update(1)
    
    pbar.close()
    
    # D. Sauvegarde finale
    lexicon_df = pd.DataFrame(all_daily_lexicons)
    lexicon_df.to_csv('lexique_quotidien_2019_2023.csv', index=False)
    return lexicon_df

news_path = '../data/processed/news_2019_2023_cleaned.csv'
price_path = '../data/raw/sp500_prices.csv'

# --- EXECUTION ---
df_final = generate_full_period_lexicons('news_2019_2023.csv', 'sp500_prices.csv')
print("Fichier 'lexique_quotidien_2019_2023.csv' généré avec succès !")

1. Chargement et nettoyage des news (patience...)


KeyError: Index(['headline'], dtype='str')